In [ ]:
import pandas as pd  

In [ ]:
data = pd.read_csv("narodni-registr-poskytovatelu-zdravotnich-sluzeb.csv")
data.info()

In [ ]:
data1 = data[["Okres","Kraj","OborPece","MistoPoskytovaniId"]]
#data1.info()

data1 = data1.rename(columns = {"Okres": 'Region',"Kraj": 'District',"OborPece":"Field_of_care",'MistoPoskytovaniId': 'Nbr_of_care_providers' })
data1.info()

In [ ]:
data1.isna().sum()

data_2 = data1.dropna(axis=0)
data_2.info()


In [ ]:
print(data_2['Region'].value_counts().count())
print(data_2['District'].value_counts().count())
print(data_2['Field_of_care'].value_counts().count())

In [ ]:
data_2 = data_2.sort_values(by=['Region','District','Field_of_care'])
data_2.head()

In [ ]:
import json

result = data_2.to_json(orient="records")
parsed = json.loads(result)
print(json.dumps(parsed, indent=4))

In [ ]:
data_2.to_json("example.json", orient = 'records')
data_2.to_csv("example.csv")


In [ ]:
#  pip install rdflib

import rdflib
from rdflib import Graph, URIRef, BNode, Literal


def load_csv_file_as_object(file_path: str):
    import csv

    result = []
    with open(file_path,encoding = "utf8") as stream:
        reader = csv.reader(stream)
        header = next(reader)  # Skip header
        for line in reader:
            result.append({key: value for key, value in zip(header, line)})
    return result

def as_rdf(content):
    result = Graph()
    prefix = "http://bastien_cadiou.org/"
    for record in content:
        resource = URIRef(f"http://bastien_cadiou.org/resource/{record['id']}")
        result.add((resource, URIRef(prefix + "region"), Literal(record["Region"])))
        result.add((resource, URIRef(prefix + "district"), Literal(record["District"])))
        result.add((resource, URIRef(prefix + "field_of_care"), Literal(record["Field_of_care"])))
        result.add((resource, URIRef(prefix + "nbr_of_care_provider"), Literal(record["Nbr_of_care_providers"])))
    return result

def print_rdf_as_trig(graph: Graph):
    print(graph.serialize(format="turtle", destination = "1st_cube.ttl"))
    
file_path = "example.csv"
data = load_csv_file_as_object(file_path)
rdf = as_rdf(data)
print_rdf_as_trig(rdf)


